In [5]:
import imp
import torch
import numpy as np
from nltk import sent_tokenize, word_tokenize
import json

In [7]:
import config.model_config
import models.vae

In [8]:
imp.reload(config.model_config)
imp.reload(models.vae)
device = torch.device("cpu")

<module 'models.vae' from '/home/chxw/Desktop/develop/maml-vae/code/models/vae.py'>

In [9]:
mconf = config.model_config.default_mconf
print(mconf.__dict__)
model = models.vae.AdvAutoencoder(device=device, mconf=mconf)

{'num_labels': 2, 'betas': [0.5, 0.999], 'grad_clip': 30.0, 'adv_loss_tolerance': 7.0, 'gamma_init': 0.001, 'gamma_min': 0.0001, 'gamma_decay': 0.9, 'min_seq_length': 5, 'max_seq_length': 15, 'vocab_size': 5000, 'vocab_cutoff': 7, 'pad_id': 0, 'eos_id': 1, 'sos_id': 2, 'unk_id': 3, 'filter_stopwords': False, 'embedding_size': 300, 'encoder_rnn_size': 128, 'decoder_rnn_size': 128, 'sentence_embedding_size': 256, 'style_embedding_size': 8, 'content_embedding_size': 120, 'sequence_word_dropout': 0.1, 'rnn_dropout': 0.1, 'fc_dropout': 0.1, 'kl_anneal_iterations': 2000, 'eps': 1e-08, 'autoencoder_train_lr': 0.001, 'adversarial_train_lr': 0.001, 'style_overall_train_lr': 0.001, 'style_kl_weight': 0.03, 'content_kl_weight': 0.03, 'style_adv_loss_weight': 1, 'content_adv_loss_weight': 0.03, 'style_multitask_loss_weight': 10, 'content_multitask_loss_weight': 3, 'corpus': 'yelp', 'last_ckpt': 'final', 'wordvec_path': None, 'model_save_dir_prefix': '../ckpt/yelp/', 'processed_data_save_dir_prefix

In [10]:
mconf.corpus = "yelp"
mconf.update_corpus()

In [11]:
import pprint
printer = pprint.PrettyPrinter(indent=4)
printer.pprint(mconf.__dict__)

{   'adv_loss_tolerance': 7.0,
    'adversarial_train_lr': 0.001,
    'autoencoder_train_lr': 0.001,
    'betas': [0.5, 0.999],
    'content_adv_loss_weight': 0.03,
    'content_embedding_size': 120,
    'content_kl_weight': 0.03,
    'content_multitask_loss_weight': 3,
    'corpus': 'yelp',
    'data_dir_prefix': '../data/yelp/',
    'decoder_rnn_size': 128,
    'embedding_size': 300,
    'encoder_rnn_size': 128,
    'eos_id': 1,
    'eps': 1e-08,
    'fc_dropout': 0.1,
    'filter_stopwords': False,
    'gamma_decay': 0.9,
    'gamma_init': 0.001,
    'gamma_min': 0.0001,
    'grad_clip': 30.0,
    'kl_anneal_iterations': 2000,
    'last_ckpt': 'final',
    'max_seq_length': 15,
    'min_seq_length': 5,
    'model_save_dir_prefix': '../ckpt/yelp/',
    'num_labels': 2,
    'output_dir_prefix': '../output/yelp/',
    'pad_id': 0,
    'processed_data_save_dir_prefix': '../data/yelp/processed/',
    'rnn_dropout': 0.1,
    'sentence_embedding_size': 256,
    'sequence_word_dropout': 0.1

In [20]:
import utils.data_loader
import utils.data_processor
import utils.vocab

imp.reload(utils.data_processor)
imp.reload(utils.data_loader)

In [13]:
vocab = utils.vocab.Vocabulary(mconf=mconf)
data_dir = "../data/yelp/"
for label in ["train", "val"]:
    for s in [0, 1]:
        vocab.update_vocab(data_dir + "{}/t1.{}".format(label, s))

lines: 10000
word: 7712, lines: 5000
word: 11177, lines: 10000
updated 2224 words
lines: 10000
word: 8160, lines: 5000
word: 11966, lines: 10000
updated 593 words
lines: 1000
updated 1 words
lines: 1000
updated 0 words


In [15]:
import pickle
with open("../data/yelp/processed/1t/vocab", "wb") as f:
    pickle.dump(vocab, f)

In [16]:
seqs = {"train": [], "val": []}
lengths = {"train": [], "val": []}
labels = {"train": [], "val": []}
bow_representations = {"train": [], "val": []}
for label in ["train", "val"]:
    s0, s1, l0, l1, lb0, lb1, bow0, bow1 = utils.data_processor.load_task_data(
        task_id=1, data_dir=data_dir, vocab=vocab,
        label=label, mconf=mconf
    )
    seqs[label] = [s0, s1]
    lengths[label] = [l0, l1]
    labels[label] = [lb0, lb1]
    bow_representations[label] = [bow0, bow1]

In [18]:
print(labels["train"][1].shape, bow_representations["train"][1].shape)
# vocab._size

(10000,) (10000, 2822)


2822

In [85]:
data = {"s": seqs, "l": lengths, "lb": labels, "bow": bow_representations}
with open("../data/yelp/processed/1t/t1", "wb") as f:
    pickle.dump(data, f)

In [21]:
batch_generator, num_batches = utils.data_processor.get_batch_generator(
    seqs=seqs["train"], lengths=lengths["train"], labels=labels["train"],
    bow_representations=bow_representations["train"], batch_size=64, device=device
)

In [23]:
imp.reload(models.vae)
# import utils.nn
imp.reload(utils.nn)
mconf.vocab_size = vocab._size
model = models.vae.AdvAutoencoder(mconf=mconf, device=device)

In [24]:
model.train(
    input_sequences_all=seqs["train"], lengths_all=lengths["train"],
    labels_all=labels["train"], bow_representations_all=bow_representations["train"],
    batch_size=64, epochs=20
)

[2019-11-30T11:28:57.436508]: batch 1/157, epoch 1/20, loss_vae 146.887, style_adversarial_loss 9.21209, content_adversarial_loss 18.3431
	 reconstruction_loss 118.056, style_kl_loss 0.235877, content_kl_loss 2.96533
	 style_multitask_loss 10.4871, content_multitask_loss 18.3445, style_overall_pred_loss 0.675156
[2019-11-30T11:28:59.217630]: batch 2/157, epoch 1/20, loss_vae 144.309, style_adversarial_loss 6.17129, content_adversarial_loss 16.6067
	 reconstruction_loss 116.372, style_kl_loss 0.225931, content_kl_loss 3.09994
	 style_multitask_loss 9.56232, content_multitask_loss 18.3748, style_overall_pred_loss 0.611632
[2019-11-30T11:29:00.982873]: batch 3/157, epoch 1/20, loss_vae 144.777, style_adversarial_loss 2.77936, content_adversarial_loss 12.1164
	 reconstruction_loss 117.183, style_kl_loss 0.315622, content_kl_loss 3.86088
	 style_multitask_loss 9.27892, content_multitask_loss 18.315, style_overall_pred_loss 0.558355
[2019-11-30T11:29:02.875628]: batch 4/157, epoch 1/20, loss

[2019-11-30T11:29:57.816745]: batch 27/157, epoch 1/20, loss_vae 96.3657, style_adversarial_loss 1.39078e-21, content_adversarial_loss 8.09277
	 reconstruction_loss 86.1761, style_kl_loss 79.4895, content_kl_loss 1405
	 style_multitask_loss 0.103941, content_multitask_loss 10.0737, style_overall_pred_loss 0.306916
[2019-11-30T11:30:00.003102]: batch 28/157, epoch 1/20, loss_vae 97.5855, style_adversarial_loss 5.54805e-19, content_adversarial_loss 8.88636
	 reconstruction_loss 87.6285, style_kl_loss 80.0698, content_kl_loss 1380.07
	 style_multitask_loss 0.099461, content_multitask_loss 9.84585, style_overall_pred_loss 0.322876
[2019-11-30T11:30:02.404708]: batch 29/157, epoch 1/20, loss_vae 96.0472, style_adversarial_loss 1.00955e-21, content_adversarial_loss 9.27658
	 reconstruction_loss 85.8525, style_kl_loss 81.5579, content_kl_loss 1478.18
	 style_multitask_loss 0.104149, content_multitask_loss 10.0779, style_overall_pred_loss 0.3133
[2019-11-30T11:30:05.110924]: batch 30/157, epoc

[2019-11-30T11:30:55.675371]: batch 53/157, epoch 1/20, loss_vae 95.3029, style_adversarial_loss 5.30661e-20, content_adversarial_loss 8.99508
	 reconstruction_loss 85.3433, style_kl_loss 108.264, content_kl_loss 1186.84
	 style_multitask_loss 0.0444256, content_multitask_loss 9.9039, style_overall_pred_loss 0.3133
[2019-11-30T11:30:57.909968]: batch 54/157, epoch 1/20, loss_vae 94.9708, style_adversarial_loss 8.93062e-21, content_adversarial_loss 10.2198
	 reconstruction_loss 85.1898, style_kl_loss 113.207, content_kl_loss 1203.53
	 style_multitask_loss 0.0519885, content_multitask_loss 9.7176, style_overall_pred_loss 0.310108
[2019-11-30T11:31:00.341299]: batch 55/157, epoch 1/20, loss_vae 97.3917, style_adversarial_loss 7.50239e-21, content_adversarial_loss 7.51026
	 reconstruction_loss 86.9171, style_kl_loss 109.449, content_kl_loss 1174.03
	 style_multitask_loss 0.0866281, content_multitask_loss 10.3768, style_overall_pred_loss 0.326068
[2019-11-30T11:31:01.486692]: batch 56/157, 

[2019-11-30T11:31:31.395813]: batch 79/157, epoch 1/20, loss_vae 96.6441, style_adversarial_loss 2.24181e-15, content_adversarial_loss 9.06329
	 reconstruction_loss 86.9305, style_kl_loss 113.939, content_kl_loss 747.235
	 style_multitask_loss 0.0442498, content_multitask_loss 9.66128, style_overall_pred_loss 0.322876
[2019-11-30T11:31:32.706181]: batch 80/157, epoch 1/20, loss_vae 96.0731, style_adversarial_loss 7.80542e-17, content_adversarial_loss 8.63288
	 reconstruction_loss 86.8933, style_kl_loss 115.07, content_kl_loss 682.328
	 style_multitask_loss 0.0705317, content_multitask_loss 9.10179, style_overall_pred_loss 0.310108
[2019-11-30T11:31:34.016137]: batch 81/157, epoch 1/20, loss_vae 96.9346, style_adversarial_loss 3.46276e-15, content_adversarial_loss 10.6465
	 reconstruction_loss 86.7687, style_kl_loss 108.968, content_kl_loss 597.63
	 style_multitask_loss 0.0752801, content_multitask_loss 10.084, style_overall_pred_loss 0.338836
[2019-11-30T11:31:35.467735]: batch 82/157,

[2019-11-30T11:32:09.879285]: batch 105/157, epoch 1/20, loss_vae 95.5032, style_adversarial_loss 5.25224e-12, content_adversarial_loss 9.89327
	 reconstruction_loss 85.6737, style_kl_loss 120.806, content_kl_loss 239.056
	 style_multitask_loss 0.0371085, content_multitask_loss 9.78873, style_overall_pred_loss 0.32926
[2019-11-30T11:32:11.884624]: batch 106/157, epoch 1/20, loss_vae 94.451, style_adversarial_loss 1.21345e-11, content_adversarial_loss 9.4811
	 reconstruction_loss 84.6858, style_kl_loss 123.459, content_kl_loss 226.03
	 style_multitask_loss 0.0169818, content_multitask_loss 9.74471, style_overall_pred_loss 0.306917
[2019-11-30T11:32:13.942441]: batch 107/157, epoch 1/20, loss_vae 95.1855, style_adversarial_loss 1.05627e-10, content_adversarial_loss 9.35792
	 reconstruction_loss 85.4042, style_kl_loss 115.851, content_kl_loss 213.678
	 style_multitask_loss 0.0315981, content_multitask_loss 9.74632, style_overall_pred_loss 0.322877
[2019-11-30T11:32:16.241377]: batch 108/1

...

[2019-11-30T11:40:50.359975]: batch 51/157, epoch 3/20, loss_vae 73.1316, style_adversarial_loss 1.45101e-06, content_adversarial_loss 9.53433
	 reconstruction_loss 70.2733, style_kl_loss 153.516, content_kl_loss 159.733
	 style_multitask_loss 0.0163599, content_multitask_loss 2.83498, style_overall_pred_loss 0.322895
[2019-11-30T11:40:52.690120]: batch 52/157, epoch 3/20, loss_vae 75.2877, style_adversarial_loss 9.90047e-07, content_adversarial_loss 9.47177
	 reconstruction_loss 72.0002, style_kl_loss 156.598, content_kl_loss 179.045
	 style_multitask_loss 0.00445398, content_multitask_loss 3.27559, style_overall_pred_loss 0.319691
[2019-11-30T11:40:55.157471]: batch 53/157, epoch 3/20, loss_vae 74.3048, style_adversarial_loss 6.74996e-07, content_adversarial_loss 9.36774
	 reconstruction_loss 71.429, style_kl_loss 156.205, content_kl_loss 168.948
	 style_multitask_loss 0.00315509, content_multitask_loss 2.8654, style_overall_pred_loss 0.338847
[2019-11-30T11:40:58.125990]: batch 54/1

[2019-11-30T11:41:38.880300]: batch 77/157, epoch 3/20, loss_vae 75.336, style_adversarial_loss 5.25973e-06, content_adversarial_loss 9.90489
	 reconstruction_loss 72.6208, style_kl_loss 151.163, content_kl_loss 150.852
	 style_multitask_loss 0.0115392, content_multitask_loss 2.69642, style_overall_pred_loss 0.332458
[2019-11-30T11:41:40.205520]: batch 78/157, epoch 3/20, loss_vae 73.2595, style_adversarial_loss 9.43691e-06, content_adversarial_loss 7.9312
	 reconstruction_loss 70.0057, style_kl_loss 152.655, content_kl_loss 173.108
	 style_multitask_loss 0.00549601, content_multitask_loss 3.24052, style_overall_pred_loss 0.329282
[2019-11-30T11:41:41.707123]: batch 79/157, epoch 3/20, loss_vae 74.0434, style_adversarial_loss 3.05097e-05, content_adversarial_loss 10.4394
	 reconstruction_loss 70.0677, style_kl_loss 141.423, content_kl_loss 143.152
	 style_multitask_loss 0.00639514, content_multitask_loss 3.96241, style_overall_pred_loss 0.31975
[2019-11-30T11:41:43.361863]: batch 80/15

[2019-11-30T11:42:16.495080]: batch 103/157, epoch 3/20, loss_vae 70.3605, style_adversarial_loss 8.89107e-07, content_adversarial_loss 8.97387
	 reconstruction_loss 67.9385, style_kl_loss 150.723, content_kl_loss 147.683
	 style_multitask_loss 0.0135441, content_multitask_loss 2.40076, style_overall_pred_loss 0.335656
[2019-11-30T11:42:17.954166]: batch 104/157, epoch 3/20, loss_vae 73.9871, style_adversarial_loss 8.5081e-05, content_adversarial_loss 10.6243
	 reconstruction_loss 70.7747, style_kl_loss 140.68, content_kl_loss 140.325
	 style_multitask_loss 0.0172879, content_multitask_loss 3.1878, style_overall_pred_loss 0.329325
[2019-11-30T11:42:20.336623]: batch 105/157, epoch 3/20, loss_vae 73.0577, style_adversarial_loss 5.85986e-06, content_adversarial_loss 10.6325
	 reconstruction_loss 70.4776, style_kl_loss 144.254, content_kl_loss 142.736
	 style_multitask_loss 0.00377765, content_multitask_loss 2.56888, style_overall_pred_loss 0.306926
[2019-11-30T11:42:22.876147]: batch 106

[2019-11-30T11:43:26.577552]: batch 129/157, epoch 3/20, loss_vae 71.4198, style_adversarial_loss 2.47822e-05, content_adversarial_loss 8.42725
	 reconstruction_loss 68.4448, style_kl_loss 132.654, content_kl_loss 154.39
	 style_multitask_loss 0.00530644, content_multitask_loss 2.96167, style_overall_pred_loss 0.329284


KeyboardInterrupt: 

In [4]:
import imp
import numpy as np
import pickle
import json
import torch
import pprint

In [2]:
import config.model_config
import models.maml_vae
import utils.data_loader
import utils.data_processor